# செலவுக் கோரிக்கை பகுப்பாய்வு

இந்த நோட்புக் ஒரு பயண செலவுகளை உள்ளூர் ரசீது படங்களிலிருந்து செயலாக்க, செலவுக் கோரிக்கை மின்னஞ்சலை உருவாக்க, மற்றும் செலவுக் தரவுகளை ஒரு வட்ட диаграмம் மூலம் காட்சிப்படுத்த பிளகின்களைப் பயன்படுத்தும் முகவர்களை உருவாக்குவது எப்படி என்பதை விளக்குகிறது. முகவர்கள் பணியின் சூழ்நிலைக்கு ஏற்ப செயல்பாடுகளை தானாகத் தேர்ந்தெடுக்கின்றன.

படிகள்:
1. OCR முகவர் உள்ளூர் ரசீது படத்தை செயலாக்கி பயண செலவுத் தரவுகளை எடுக்கிறது.
2. மின்னஞ்சல் முகவர் செலவுக் கோரிக்கை மின்னஞ்சலை உருவாக்குகிறது.

### ஒரு பயண செலவுத் சூழ்நிலையின் உதாரணம்:
நீங்கள் ஒரு ஊழியராக மற்றொரு நகரத்தில் வணிகக் கூட்டத்திற்காக பயணம் செய்கிறீர்கள் என்று கற்பனை செய்யுங்கள். உங்கள் நிறுவனம் அனைத்து நியாயமான பயண தொடர்பான செலவுகளை ஈடு செய்யும் கொள்கையை கொண்டுள்ளது. இங்கே சாத்தியமான பயண செலவுகளின் விவரக்குறிப்பு உள்ளது:
- **போக்குவரத்து:**
உங்கள் வீட்டுப் நகரத்திலிருந்து இலக்கு நகரத்திற்கு சுற்றுப்பயண விமானக் கட்டணம்.
விமான நிலையத்திற்கும், விமான நிலையத்திலிருந்து டாக்ஸி அல்லது ரைடு-ஹெய்லிங் சேவைகள்.
இலக்கு நகரத்தில் உள்ள உள்ளூர் போக்குவரத்து (பொது போக்குவரத்து, வாடகை கார்கள், அல்லது டாக்ஸிகள் போன்றவை).

- **வசதி:**
கூட்டம் நடைபெறும் இடத்திற்கு அருகிலுள்ள நடுத்தர வர்க்க வணிக ஹோட்டலில் மூன்று இரவுகள் தங்கும் செலவு.

- **உணவு:**
நிறுவனத்தின் தினசரி உணவுக்கான கொள்கையின் அடிப்படையில் காலை உணவு, மதிய உணவு, மற்றும் இரவு உணவுக்கான தினசரி அனுமதி.

- **பொது செலவுகள்:**
விமான நிலையத்தில் பார்க்கிங் கட்டணங்கள்.
ஹோட்டலில் இணைய அணுகல் கட்டணங்கள்.
உதவிக்கட்டணங்கள் அல்லது சிறிய சேவை கட்டணங்கள்.

- **ஆவணங்கள்:**
நீங்கள் அனைத்து ரசீதுகளை (விமானங்கள், டாக்ஸிகள், ஹோட்டல், உணவுகள், போன்றவை) மற்றும் பூர்த்தி செய்யப்பட்ட செலவுக் அறிக்கையை ஈடுசெய்ய சமர்ப்பிக்கிறீர்கள்.


## தேவையான நூலகங்களை இறக்குமதி செய்யவும்

நூட்ட்புக்கிற்கான தேவையான நூலகங்கள் மற்றும் தொகுதிகளை இறக்குமதி செய்யவும்.


In [1]:
import os
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from openai import AsyncOpenAI
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat


from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents.strategies import SequentialSelectionStrategy, DefaultTerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents import ImageContent, TextContent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings

from semantic_kernel.functions import kernel_function, KernelArguments
from pydantic import BaseModel, Field
from typing import List
from azure.ai.inference.models import SystemMessage, UserMessage, TextContentItem, ImageContentItem, ImageUrl, ImageDetailLevel

load_dotenv()

True

In [2]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
   
    client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"], base_url="https://models.inference.ai.azure.com/")
    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o-mini",
            async_client=client,
            service_id="open_ai"
        )
    )

    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o",
            async_client=client,
            service_id="gpt-4o"
        )
    )

    return kernel

## செலவுக் மாதிரிகளை வரையறுக்கவும்

தனிப்பட்ட செலவுகளுக்கான Pydantic மாதிரியை உருவாக்கவும் மற்றும் பயனர் கேள்வியை அமைவான செலவுத் தரவாக மாற்ற ExpenseFormatter வகுப்பை உருவாக்கவும்.

ஒவ்வொரு செலவும் கீழே உள்ள வடிவத்தில் பிரதிநிதித்துவம் செய்யப்படும்:
`{'date': '07-Mar-2025', 'description': 'destination-க்கு விமானம்', 'amount': 675.99, 'category': 'Transportation'}`


In [3]:
class Expense(BaseModel):
    date: str = Field(..., description="Date of expense in dd-MMM-yyyy format")
    description: str = Field(..., description="Expense description")
    amount: float = Field(..., description="Expense amount")
    category: str = Field(..., description="Expense category (e.g., Transportation, Meals, Accommodation, Miscellaneous)")

class ExpenseFormatter(BaseModel):
    raw_query: str = Field(..., description="Raw query input containing expense details")
    
    def parse_expenses(self) -> List[Expense]:
        """
        Parses the raw query into a list of Expense objects.
        Expected format: "date|description|amount|category" separated by semicolons.
        """
        expense_list = []
        for expense_str in self.raw_query.split(";"):
            if expense_str.strip():
                parts = expense_str.strip().split("|")
                if len(parts) == 4:
                    date, description, amount, category = parts
                    try:
                        expense = Expense(
                            date=date.strip(),
                            description=description.strip(),
                            amount=float(amount.strip()),
                            category=category.strip()
                        )
                        expense_list.append(expense)
                    except ValueError as e:
                        print(f"[LOG] Parse Error: Invalid data in '{expense_str}': {e}")
        return expense_list

## முகவரிகளை வரையறுத்தல் - மின்னஞ்சலை உருவாக்குதல்

செலவுக் கோரிக்கையை சமர்ப்பிக்க மின்னஞ்சல் உருவாக்க ஒரு முகவர் வகுப்பை உருவாக்குங்கள்.
- இந்த முகவர் `kernel_function` அலங்காரத்தை பயன்படுத்தி செலவுக் கோரிக்கையை சமர்ப்பிக்க மின்னஞ்சல் உருவாக்கும் ஒரு செயல்பாட்டை வரையறுக்கிறது.
- இது செலவுகளின் மொத்த தொகையை கணக்கிடுகிறது மற்றும் விவரங்களை மின்னஞ்சல் உடலாக வடிவமைக்கிறது.


In [4]:
class ExpenseEmailAgent:

    @kernel_function(description="Generate an email to submit an expense claim to the Finance Team")
    async def generate_expense_email(expenses):
        total_amount = sum(expense['amount'] for expense in expenses)
        email_body = "Dear Finance Team,\n\n"
        email_body += "Please find below the details of my expense claim:\n\n"
        for expense in expenses:
            email_body += f"- {expense['description']}: ${expense['amount']}\n"
        email_body += f"\nTotal Amount: ${total_amount}\n\n"
        email_body += "Receipts for all expenses are attached for your reference.\n\n"
        email_body += "Thank you,\n[Your Name]"
        return email_body

# ரசீது படங்களிலிருந்து பயண செலவுகளை எடுக்க ஒரு முகவர்

ரசீது படங்களிலிருந்து பயண செலவுகளை எடுக்க ஒரு முகவர் வகுப்பை உருவாக்கவும்.
- இந்த முகவர் `kernel_function` அலங்காரத்தை பயன்படுத்தி ரசீது படங்களிலிருந்து பயண செலவுகளை எடுக்கும் ஒரு செயல்பாட்டை வரையறுக்கிறது.
- ரசீது படத்தை OCR (ஒளியியல் எழுத்து அங்கீகாரம்) மூலம் உரையாக மாற்றி, தேதி, விளக்கம், தொகை மற்றும் வகை போன்ற தொடர்புடைய தகவல்களை எடுக்கவும்.


In [5]:
class OCRAgentPlugin:
    def __init__(self):
        self.client = ChatCompletionsClient(
            endpoint="https://models.inference.ai.azure.com/",
            credential=AzureKeyCredential(os.environ.get("GITHUB_TOKEN")),
        )
        self.model_name = "gpt-4o"

    @kernel_function(description="Extract structured travel expense data from receipt.jpg using gpt-4o-model")
    def extract_text(self, image_path: str = "receipt.jpg") -> str:
        try:
            image_url_str = str(ImageUrl.load(image_file=image_path, image_format="jpg", detail=ImageDetailLevel.HIGH))

            prompt = (
                "You are an expert OCR assistant specialized in extracting structured data from receipt images. "
                "Analyze the provided receipt image and extract travel-related expense details in the format: "
                "'date|description|amount|category' separated by semicolons. "
                "Follow these rules: "
                "- Date: Convert dates (e.g., '4/4/22') to 'dd-MMM-yyyy' (e.g., '04-Apr-2022'). "
                "- Description: Extract item names (e.g., 'Carlson's Drylawn', 'Peigs transaction Probiotics'). "
                "- Amount: Use numeric values (e.g., '4.50' from '$4.50' or '4.50 dollars'). "
                "- Category: Infer from context (e.g., 'Meals' for food, 'Transportation' for travel, 'Accommodation' for lodging, 'Miscellaneous' otherwise). "
                "Ignore totals, subtotals, or service charges unless they are itemized expenses. "
                "If no expenses are found, return 'No expenses detected'. "
                "Return only the structured data, no additional text."
            )
            response = self.client.complete(
                messages=[
                    SystemMessage(content=prompt),
                    UserMessage(content=[
                        TextContentItem(text="Extract travel expenses from this receipt image."),
                        ImageContentItem(image_url=ImageUrl(url=image_url_str))
                    ])
                ],
                model=self.model_name,
                temperature=0.1,
                max_tokens=2048
            )
            extracted_text = response.choices[0].message.content
            return extracted_text
        except Exception as e:
            error_msg = f"[LOG] OCR Plugin: Error processing image: {str(e)}"
            print(error_msg)
            return error_msg

## செலவுகளை செயலாக்குதல்

செலவுகளை செயலாக்க தேவையான முகவர்களை உருவாக்கி பதிவு செய்து, பின்னர் அவற்றை அழைக்கும் ஒரு அசிங்க்ரோனஸ் செயல்பாட்டை வரையறுக்கவும்.
- இந்த செயல்பாடு செலவுகளை செயலாக்குவதற்கு சூழல் மாறிகளை ஏற்றுகிறது, தேவையான முகவர்களை உருவாக்குகிறது மற்றும் அவற்றை பிளகின்களாக பதிவு செய்கிறது.
- இது இரண்டு முகவர்களுடன் ஒரு குழு உரையாடலை உருவாக்கி, செலவுகள் தரவின் அடிப்படையில் மின்னஞ்சல் மற்றும் பை சார்ட் உருவாக்க ஒரு உந்துதல் செய்தியை அனுப்புகிறது.
- உரையாடல் அழைப்பின் போது ஏற்படும் எந்தவொரு பிழைகளையும் கையாளுகிறது மற்றும் முகவர்களின் சரியான சுத்தமாக்கலை உறுதிப்படுத்துகிறது.


In [6]:
async def process_expenses():
    load_dotenv()
    settings_slm = OpenAIChatPromptExecutionSettings(service_id="gpt-4o")
    settings_llm = OpenAIChatPromptExecutionSettings(service_id="open_ai")  # Fixed typo in service_id
    
    ocr_agent = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion("ocrAgent"),
        name="ocr_agent",
        instructions="Extract travel expense data from the receipt image in the prompt using the 'extract_text' function from the 'ocrAgent' plugin. Return the data in the format 'date|description|amount|category' separated by semicolons.",
        arguments=KernelArguments(settings=settings_slm)
    )
    
       
    email_agent = ChatCompletionAgent(
            kernel=_create_kernel_with_chat_completion("expenseEmailAgent"),
            name="email_agent",
            instructions="Take the travel expense data from the previous agent and generate a professional expense claim email using the 'generate_expense_email' function from the 'expenseEmailAgent' plugin, then pass the data forward.",
            arguments=KernelArguments(
                settings=settings_llm)
        )


    kernel = Kernel()

    # Use fixed path to receipt.jpg in the same folder
    image_path = "./receipt.jpg"
    
    # Create a structured message with text and image content for OCR processing
    image_url_str = f"file://{image_path}"
    
    # Using the correct format for multi-modal content
    user_message = ChatMessageContent(
        role=AuthorRole.USER,
        items=[
            TextContent(text="""
            Please extract the raw text from this receipt image, focusing on travel expenses like dates, descriptions, amounts, and categories (e.g., Transportation, Accommodation, Meals, Miscellaneous).
            Then generate a professional expense claim email.
                        """),
            ImageContent.from_image_file(path=image_path)
        ]
    )

    # Register plugins with the kernel
    kernel.add_plugin(OCRAgentPlugin(), plugin_name="ocrAgent")
    kernel.add_plugin(ExpenseEmailAgent(), plugin_name="expenseEmailAgent")

    # Create group chat
    chat = AgentGroupChat(
        agents=[ocr_agent, email_agent],
        selection_strategy=SequentialSelectionStrategy(initial_agent=ocr_agent),
        termination_strategy=DefaultTerminationStrategy(maximum_iterations=1)
    )

    # Add user message with prompt
    await chat.add_chat_message(user_message)
    print(f"# User message added to chat with receipt image")

    async for content in chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")


## முக்கிய செயல்பாடு

முக்கிய செயல்பாட்டை வரையறுக்கவும், இது கன்சோலை அழித்து, `process_expenses` செயல்பாட்டை அசிங்கமாக இயக்கும்.


In [9]:
async def main():
    # Clear the console
    os.system('cls' if os.name=='nt' else 'clear')

    # Run the async agent code
    await process_expenses()

await main()

# User message added to chat with receipt image
# Agent - ocr_agent: 'The receipt primarily seems to capture costs for meals and beverages. Below is the extracted travel expense data:

**Travel Expense Data:**  
`2 May '22|Meals at restaurant|75.15|Meals`

---

**Professional Expense Claim Email Draft:**  

**Subject:** Expense Claim for Meals – 2 May 2022  

Dear [Recipient's Name],  

I am submitting an expense claim for a meal incurred during a business-related trip. Below are the details:  

- **Date:** 2 May 2022  
- **Expense Description:** Meals at a restaurant  
- **Amount:** $75.15  
- **Category:** Meals  

Please find the attached receipt for your reference. Kindly process the reimbursement at your earliest convenience. Let me know if you require additional information.  

Thank you for your assistance.  

Best regards,  
[Your Name]  
[Your Contact Information]  

Let me know if you need further revisions or additional details!'



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கின்றோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளுங்கள். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
